In [1]:
!pip install streamlit transformers torch pyngrok pandas --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [6]:
%%writefile app.py
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

import streamlit as st
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load sentiment model
sentiment_model_name = "cardiffnlp/twitter-roberta-base-sentiment"
sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model_name)
sentiment_model = AutoModelForSequenceClassification.from_pretrained(sentiment_model_name)
sentiment_model.eval()

sentiment_labels = {0: "Negative", 1: "Neutral", 2: "Positive"}

# Department mapping
department_keywords = {
    "loan process": ["loan", "home loan", "personal loan", "loan process"],
    "branch experience": ["branch", "staff", "manager", "in person"],
    "customer service": ["customer service", "support", "call", "response"],
    "account opening": ["account opening", "new account", "open account"],
    "mobile app": ["app", "application", "mobile", "online"],
    "charges/fees": ["charges", "fees", "hidden charge", "deduction"],
    "credit card": ["credit card", "card limit", "card issue"]
}

def predict_sentiment(text):
    inputs = sentiment_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = sentiment_model(**inputs)
    pred = torch.argmax(outputs.logits, dim=1).item()
    return sentiment_labels[pred]

def predict_service(text):
    text_lower = text.lower()
    for dept, keywords in department_keywords.items():
        for kw in keywords:
            if kw in text_lower:
                return dept
    return "other"

# --- Streamlit UI ---
st.title("Sentiment Analysis App")
st.write("Upload a `.csv` file with **two columns: City, Feedback**")

uploaded_file = st.file_uploader("Upload CSV", type=["csv"])

if uploaded_file is not None:
    df = pd.read_csv(uploaded_file)
    if "City" not in df.columns or "Feedback" not in df.columns:
        st.error("CSV must contain 'City' and 'Feedback' columns.")
        st.stop()

    df = df.dropna(subset=["City", "Feedback"])
    df["Sentiment"] = df["Feedback"].apply(predict_sentiment)
    df["Department"] = df["Feedback"].apply(predict_service)

    st.write("### 🔍 Prediction Results")
    st.dataframe(df)

    st.write("### 📊 Sentiment Summary by Department (All Cities)")
    dept_summary = pd.crosstab(df["Department"], df["Sentiment"])
    st.dataframe(dept_summary)
    st.bar_chart(dept_summary)

    st.write("### 🏙️ City-wise Department Sentiment Breakdown")
    grouped = df.groupby(["City", "Department", "Sentiment"]).size().unstack(fill_value=0)
    st.dataframe(grouped)

    st.write("### 🌍 Summary: Sentiment Distribution by City")
    city_summary = pd.crosstab(df["City"], df["Sentiment"])
    st.dataframe(city_summary)
    st.bar_chart(city_summary)

Overwriting app.py


In [7]:
from pyngrok import ngrok

# Kill all active tunnels
ngrok.kill()


In [8]:
from pyngrok import ngrok
import time

# Set your authtoken
ngrok.set_auth_token("2zdLIdrYO2ChVzZ2tpYojT5k57z_2NBZ7wkjNskTzDaHEhZLf")

# Restart the Streamlit app
get_ipython().system_raw("streamlit run app.py &")

# Wait a moment to ensure it's started
time.sleep(3)

# Start a fresh tunnel
public_url = ngrok.connect(8501)
print("✅ Streamlit App is live at:", public_url)


✅ Streamlit App is live at: NgrokTunnel: "https://2a3591e1b6a4.ngrok-free.app" -> "http://localhost:8501"
